# Reetan YOLO_DenseNet_V1 -> V2


**Mallien lataus**

Lataa YOLO-malli

yolo_model = YOLO('runs/detect/train_yolo_7afd0_00001/weights/best.pt')

Laita polkuun oma paras mallisi



In [ ]:
# %% [Lataukset ja asetukset]
%matplotlib inline
import torch
import cv2
import os
import glob
import random
import logging
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter
from PIL import Image
import csv
import yaml

from torchvision import datasets, transforms, models
from torch import nn
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
from ultralytics import YOLO

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Käytetään laitetta: {device}")

# %% [Mallien lataus]
# Lataa YOLO-malli
yolo_model = YOLO('runs/detect/train_yolo_7afd0_00001/weights/best.pt')

# Lataa DenseNet + muunnos
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

sample_dataset = datasets.ImageFolder('datasets/materials/cropped/train', transform=transform)
class_names_from_folder = sample_dataset.classes

densenet_model = models.densenet121(weights=None)
densenet_model.classifier = nn.Linear(densenet_model.classifier.in_features, len(class_names_from_folder))
densenet_model.load_state_dict(torch.load('densenet_materials_best.pth', map_location=device))
densenet_model.to(device)
densenet_model.eval()

# %% [YOLO + DenseNet -kuvanäyttö]
def show_yolo_vs_densenet(image_path, yolo_model, densenet_model, class_names):
    global bbox_crop_labels

    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Kuvaa ei löydy: {image_path}")
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    yolo_img, densenet_img, hybrid_img = img.copy(), img.copy(), img.copy()

    results = yolo_model(image_path, conf=0.25)[0]
    font = cv2.FONT_HERSHEY_SIMPLEX

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        yolo_cls = int(box.cls[0])
        yolo_conf = float(box.conf[0])

        # Tallenna YOLO-luokan nimi tilastointia varten
        bbox_crop_labels.append(class_names[yolo_cls])

        crop = densenet_img[y1:y2, x1:x2]
        if crop.size == 0:
            continue
        pil_crop = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
        input_tensor = transform(pil_crop).unsqueeze(0).to(device)

        with torch.no_grad():
            output = densenet_model(input_tensor)
            pred_cls = output.argmax(dim=1).item()
            dn_conf = torch.softmax(output, dim=1)[0][pred_cls].item()

        for target_img, label, color in [
            (yolo_img, f"{class_names[yolo_cls]} ({yolo_conf*100:.1f}%)", (0, 0, 0)),
            (densenet_img, f"{class_names[pred_cls]} ({dn_conf*100:.1f}%)", (0, 0, 0)),
        ]:
            cv2.rectangle(target_img, (x1, y1), (x2, y2), color, 3)
            (tw, th), _ = cv2.getTextSize(label, font, 1.2, 2)
            cv2.putText(target_img, label, (x1, max(y1 - 10, th)), font, 1.2, color, 2)

        if yolo_conf >= 0.7:
            final_cls, final_conf, source = yolo_cls, yolo_conf, "YOLO"
            color = (0, 0, 0)
        else:
            final_cls, final_conf, source = pred_cls, dn_conf, "DenseNet"
            color = (0, 0, 0)

        label = f"{class_names[final_cls]} ({final_conf*100:.1f}%) [{source}]"
        cv2.rectangle(hybrid_img, (x1, y1), (x2, y2), color, 2)
        (tw, th), _ = cv2.getTextSize(label, font, 1.2, 2)
        cv2.putText(hybrid_img, label, (x1, max(y1 - 10, th)), font, 1.2, color, 2)

    # Näytä kuvat
    plt.figure(figsize=(18, 6))
    for idx, (img, title) in enumerate(zip(
        [yolo_img, densenet_img, hybrid_img],
        ["YOLO-luokitus", "DenseNet-luokitus", "Hybridiluokitus"]
    )):
        plt.subplot(1, 3, idx + 1)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(title, fontsize=18)
        plt.axis("off")
    plt.tight_layout()
    plt.show()

# %% [Lue YOLO:n class-nimet .yaml:sta]
with open('config/materials.yaml', 'r') as f:
    config = yaml.safe_load(f)

names_dict = config['names']
print("YOLO-luokat:", ' '.join(f"{idx}: {name}" for idx, name in names_dict.items()))
print("DenseNet-luokat:", class_names_from_folder)

# %% [YOLO-analyysi koko validointidatalle]
bbox_crop_labels = []

image_files = glob.glob('datasets/materials/images/validation/*.jpg') + \
              glob.glob('datasets/materials/images/validation/*.png')

print(f"\n🔎 Käsitellään {len(image_files)} validointikuvaa YOLO-luokkien tilastoihin...")

for path in tqdm(image_files, desc="Kerätään YOLO-luokkia"):
    img = cv2.imread(path)
    if img is None:
        continue
    results = yolo_model(path, conf=0.25)[0]
    for box in results.boxes:
        yolo_cls = int(box.cls[0])
        bbox_crop_labels.append(names_dict[yolo_cls])


# %% [Näytä 3 satunnaista kuvaa]
print("\n📷 Näytetään 3 satunnaista validointikuvaa YOLO + DenseNet -luokituksilla:")
random_images = random.sample(image_files, 3)

for path in random_images:
    show_yolo_vs_densenet(path, yolo_model, densenet_model, class_names_from_folder)

# %% [Analysoi ja tallenna tulokset]
crop_counter = Counter(bbox_crop_labels)
total_crops = sum(crop_counter.values())
all_possible_classes = ['Betoni', 'Ei materiala', 'Materiaali ei tiedossa', 'Muovi', 'Teräs']

print(f"\n📦 YOLO löysi yhteensä {total_crops} kohdetta (croppeja).")
print("📊 Cropit per luokka:")
for cls in all_possible_classes:
    print(f" - {cls:25s}: {crop_counter.get(cls, 0)}")

missing_classes = [cls for cls in all_possible_classes if crop_counter.get(cls, 0) == 0]
if missing_classes:
    print("\n⚠️ Luokat, joita YOLO ei löytänyt ollenkaan:")
    for cls in missing_classes:
        print(f" - {cls}")
else:
    print("\n✅ Kaikki luokat löytyivät vähintään kerran.")

csv_path = "yolo_crop_stats.csv"
with open(csv_path, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Luokka', 'Määrä'])
    for cls in all_possible_classes:
        writer.writerow([cls, crop_counter.get(cls, 0)])
print(f"\n📁 Tiedot tallennettu tiedostoon: {csv_path}")

# %% [Visualisoi]
plt.figure(figsize=(10, 6))
plt.bar(all_possible_classes, [crop_counter.get(cls, 0) for cls in all_possible_classes], color='skyblue')
plt.title("YOLO:n tunnistamat cropit per luokka (kaikki validointikuvat)")
plt.xlabel("Luokka")
plt.ylabel("Lukumäärä")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
# %% [DenseNet-tarkkuus koko validointidatalla]
print("\n🧪 Lasketaan DenseNetin tarkkuus koko validointidatalle...")

# Lataa validointidata
val_data = datasets.ImageFolder('datasets/materials/cropped/validation', transform=transform)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

# Arvioi
densenet_model.eval()
true_labels = []
predicted_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = densenet_model(images)
        preds = outputs.argmax(dim=1)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(preds.cpu().numpy())

# Tee DenseNetin luokkien nimilista järjestyksessä
dense_class_names = [name for idx, name in sorted(names_dict.items())]

# Tulosta tarkkuusluvu
print("\n📊 DenseNetin tarkkuus (validation-cropit):")
print(classification_report(
    true_labels, 
    predicted_labels, 
    labels=list(range(len(dense_class_names))), 
    target_names=dense_class_names
))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(true_labels, predicted_labels)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, xticklabels=dense_class_names, yticklabels=dense_class_names, fmt='d')
plt.xlabel('Ennustettu')
plt.ylabel('Todellinen')
plt.title('Confusion Matrix: DenseNet')
plt.show()
